In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

#pip install bs4
#pip install selenium
#pip install webdriver_manager
#pip install blinker==1.6.2

In [16]:
SCROLL_PAUSE_TIME = 3
MAX_SCROLLS = 5
pages = int(42)
cards = []

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
service = Service(ChromeDriverManager().install())

for i in range(1, pages+1):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    url = "https://www.zapimoveis.com.br/venda/apartamentos/go+goiania++setor-marista/?pagina=" + str(i)
    driver.get(url)

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_count = 0
    while scroll_count < MAX_SCROLLS:
        img_element = driver.find_element(By.XPATH, "//img[contains(@class, 'campaign__container-image')]")
        driver.execute_script("arguments[0].scrollIntoView();", img_element)
        
        time.sleep(SCROLL_PAUSE_TIME)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        scroll_count += 1

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    anuncios = soup.find('div', class_="listing-wrapper__content").find_all('div', class_="BaseCard_card__content__pL2Vc w-full p-3")

    for anuncio in anuncios:
        card = {}
        lista_valores = anuncio.find_all('p')
        valor_limpo = [valor.text.strip() for valor in lista_valores]
        
        card['Endereço'] = valor_limpo[0]
        card['Preço'] = valor_limpo[1].replace("A partir de ", "")

        lista_carac = anuncio.find_all('li')
        carac_limpo = [carac.text.strip() for carac in lista_carac]

        card['Área (m2)'] = carac_limpo[0].replace(" m²", "")
        card['Quartos'] = carac_limpo[1]
        card['Banheiros'] = carac_limpo[2]
        card['Garagem'] = carac_limpo[3] if len(carac_limpo) > 3 else 0

        cards.append(card)

    driver.quit()


In [17]:
#pip install pandas
import pandas as pd

df = pd.DataFrame(cards)
df

,Endereço,Preço,Área (m2),Quartos,Banheiros,Garagem
0,Rua 1141,R$ 800.000,80,2,3,1
1,Rua 1141,R$ 4.190.000,354,4,5,3
2,Rua 146,R$ 1.350.000,138,3,4,2
3,Rua 38,R$ 1.600.000,130,3,4,2
4,Rua 1123,R$ 420.000,45,1,1,1
...,...,...,...,...,...,...
8646,,R$ 4.600.000,306,4,6,5
8647,,R$ 1.180.000,111,3,5,2
8648,,R$ 2.200.000,208,3,1,3
8649,,R$ 1.260.000,137,3,1,2


In [18]:
df.value_counts()

Endereço                   Preço         Área (m2)  Quartos  Banheiros  Garagem
Rua 1124                   R$ 1.500.000  125        3        4          2          42
Rua 143                    R$ 930.000    115        3        7          2          41
Rua 24                     R$ 1.800.000  157        3        2          3          35
Rua 146                    R$ 1.500.000  137        3        4          2          31
Rua Amélia Artiaga Jardim  R$ 1.230.000  116        3        4          2          30
                                                                                   ..
Rua T 53                   R$ 710.000    60         2        3          2           1
                           R$ 8.609.423  509        4        5          5           1
                           R$ 8.728.938  538        4        6          6           1
                           R$ 1.790.000  150        3        5          2           1
                           R$ 1.050.000  109-113    3       

In [19]:
df = df.drop_duplicates()
df.value_counts()

Endereço    Preço         Área (m2)  Quartos  Banheiros  Garagem
Viela 1140  R$ 940.000    116        3        3          2          1
Rua T 55    R$ 400.000    28         1        1          0          1
            R$ 380.000    28         1        1          1          1
                                                         0          1
            R$ 360.000    28         1        1          0          1
                                                                   ..
            R$ 1.050.000  115        3        3          2          1
                          111        3        4          2          1
                          109-113    3        4          2          1
                          106        3        2          2          1
                          104        3        4          2          1
Name: count, Length: 2385, dtype: int64

In [20]:
df.to_csv("ApartamentosMarista.csv", index=False, sep=";", encoding="latin1")